### Simple convolutional neural network 

Data by @RDizzl3 https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/304686

WIP 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
import glob
import matplotlib.pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Reshape,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import pandas as pd
df = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Get the train image file names
# Use the * as a wild card this will tell glob get us all images in this directory
image_path = '/kaggle/input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128/*'
train_filenames = glob.glob(image_path)

In [ ]:
# Get a single image path
image_path = train_filenames[0]
img = cv2.imread(image_path)

# Check the shape - should be (128, 128, 3)
img.shape

In [ ]:
# Plot the image
plt.imshow(img)

In [ ]:
df.head()

In [ ]:

train_df = df[['image','individual_id']]

In [ ]:
train_df.head()

In [ ]:
callbacks = [
            EarlyStopping(patience = 10,monitor="accuracy",mode="max"),    
            ]

In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1
                                   )

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [ ]:
train_dir = '/kaggle/input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128/'

In [ ]:
df.head()

In [ ]:
train_df.head()

In [ ]:
train_dir

In [ ]:
individuals = list(df['individual_id'].unique())

In [ ]:
training_set = train_datagen.flow_from_dataframe(
                                                train_df,
                                                train_dir,
                                                seed=101,                                                 
                                                target_size=(64, 64),
                                                labels = individuals,
                                                batch_size=32,
                                                x_col='image',
                                                y_col='individual_id',
                                                class_mode='categorical',
                                                validation_split=0.2)

In [ ]:
validation_set = validation_datagen.flow_from_dataframe(
                                                train_df,
                                                train_dir,
                                                seed=101,                                                 
                                                target_size=(64, 64),
                                                labels = individuals,
                                                batch_size=32,
                                                x_col='image',
                                                y_col='individual_id',
                                                class_mode='categorical',
                                                subset = "validation"
                                                )

In [ ]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)

In [ ]:
x = data_augmentation(inputs)  # Apply random data augmentation
x = tf.keras.applications.xception.preprocess_input(x)

In [ ]:
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(15587)(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.Accuracy()])
model.fit(training_set, epochs=10, validation_data=validation_set)

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.Accuracy()])

# Train end-to-end. Be careful to stop before you overfit!
history = model.fit(training_set, epochs=10,validation_data=validation_set)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
import pandas as pd
losses = pd.DataFrame(history.history)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.xlabel('Epochs')
plt.ylabel('Loss')
losses['loss'].plot(label='Training Loss')
losses['val_loss'].plot(label='Validation Loss')
plt.legend(loc='center')

In [ ]:
loss, accuracy = model.evaluate(validation_set)
print('Test accuracy :', accuracy)

It doesn't seem to work!